Introduction to Bokeh
=====================

In this tutorial we'll learn how to use Bokeh to build interactive visualizations viewable in a browser.  Generally this tutorial will have the following format

`plotting` - Intermediate interface allowing control to all parts of a plot
    - Vectorized attributes
    - Toolbars
    - Linked Brushing
    - Interactivity

Install
-------

This tutorial uses many different libraries that are all available with the [Anaconda Distribution](http://continuum.io/downloads). Once you have Anaconda install, please run these commands from a terminal:

```
$ conda install -y bokeh
```


<hr/>


# Introduction to [Bokeh](http://bokeh.pydata.org/)

Provide a first-class visualization library for web-aware applications, without
requiring web-level programming.

Write a visualization python. Bokeh creates data descripors and a scenegraph consumed by BokehJS.
This works in ipython notebook, creating static files and interacting with dynamic data sources.

Bokeh includes pre-built schemas in bokeh.charts, a low-level composition interface (similar to matplotlib),
a server for large and/or dynamic datasources and widgets for providing client-side realtime interaction.

The non-JS framework also has prototypes in other languages (Scala, Julia...maybe R).

**Note:** There are examples notebooks in bokeh/examples/plotting/notebooks. Start an ipython notebook server there to get more examples.

[Gallery](http://bokeh.pydata.org/docs/gallery.html) --
[tutorial](http://bokeh.pydata.org/tutorial/index.html) -- 
[Documentation](http://bokeh.pydata.org/docs/user_guide.html) --
[Repo](https://github.com/bokeh/bokeh)


In [1]:
import pandas as pd
import numpy as np

## Output

Bokeh can output to html, a notebook, or just fragments for embedding in a web application. 

To start playing, we'll use the notebook. Later we will see the other types of output.

In [2]:
from bokeh.plotting import output_notebook  
output_notebook() # Tell Bokeh to output in an ipython notebook (other options later)

Loading BokehJS ...

<hr/>

## Plotting.py

This is a mid-level interface, used by the charting library (and other parts of bokeh).   It can also be used directly.
The basic idea: there is an active plot, and you are modifying it.


In [5]:
import numpy as np
from bokeh.plotting import *
N = 1002
lin_arr = np.linspace(0, 4*np.pi, N)
sin_arr = np.sin(lin_arr)
cos_arr = np.cos(lin_arr)

<hr />

## Scatter Plots

To begin with let's make a scatter plot.

In [6]:

p1 = figure()
p1.scatter(lin_arr, sin_arr, color="#FF00FF")
p1.scatter(lin_arr, cos_arr, color="green")
show(p1)

<hr />

## Exercise

Play with plotting arrays, try editting
- color,
- markers,
- alpha (value between 0-1), and 
- size (int of pixels)

In [5]:
p2 = figure()
p2.scatter(x=lin_arr, y=sin_arr, color="red")
show(p2)

In [6]:
p3 = figure()
p3.scatter(x=lin_arr, y=cos_arr , marker="square", color="green")
show(p3)

### Other plotting things...
There are lots of [glyph types](http://bokeh.pydata.org/docs/reference/plotting.html) and lots of [properties](http://bokeh.pydata.org/docs/user_guide/objects.html#userguide-objects-line-properties)...here is just a sample

<hr />
## Everything is vectorized

While we have only been passing vectors for locations, we can do so for almost any parameter.

Let's use the `cos_arr` to size the circles

In [10]:
p4 = figure()
p4.scatter(x=lin_arr, y=sin_arr, size=cos_arr**2)
show(p4)

<hr/>

Let's play with colors now. Brewer is a popular set of palletes. Here we pick one and then build a vector of colors for the plot.

In [11]:
from bokeh.palettes import brewer
print("Brewer Palettes:", brewer.keys())
print("Brewer Grey Palettes:", brewer["Greys"].keys())
palette = brewer["Greys"][9] + list(reversed(brewer["Greys"][9]))
colors = palette * int(len(lin_arr) / len(palette)) + palette[0:len(lin_arr) % len(palette)]

Brewer Palettes: dict_keys(['YlOrBr', 'PiYG', 'Greys', 'YlGnBu', 'RdPu', 'BuPu', 'PuRd', 'PRGn', 'GnBu', 'PuBuGn', 'RdBu', 'Blues', 'PuOr', 'BuGn', 'RdYlGn', 'YlOrRd', 'Greens', 'Purples', 'RdGy', 'OrRd', 'Spectral', 'PuBu', 'BrBG', 'RdYlBu', 'Oranges', 'YlGn', 'Reds'])
Brewer Grey Palettes: dict_keys([3, 4, 5, 6, 7, 8, 9])


In [12]:
p5 = figure()
p5.scatter(x=lin_arr, y=sin_arr, size=cos_arr**2*10 + 5, fill_color=colors)
show(p5)

<hr/>

## Tools

If you notice the bar at the top of the you see several places to interact with the plot.

These are tools and there a many different tools built into the Bokeh.

Let's take a look at HoverTools, but first we use Bokeh's data source which watches changes.

In [10]:
source = ColumnDataSource(
    data=dict(
        x=lin_arr,
        y=sin_arr,
        size=cos_arr**2*10 + 5,
        colors=colors
    )
)

In [11]:
from bokeh.models import HoverTool
from collections import OrderedDict

TOOLS="crosshair,pan,wheel_zoom,box_zoom,reset,hover,previewsave"
p6 = figure(title="Hoverful Scatter", tools=TOOLS)
p6.circle(x="x", y="y", size="size", source=source,
    fill_color="colors", fill_alpha=0.6, line_color=None)

hover = p6.select(dict(type=HoverTool))
hover.tooltips = OrderedDict([
    ("index", "$index"),
    ("(x,y)", "(@x, @y)"),
    ("size", "@size"),
    ("fill color", "$color[hex, swatch]:fill_color"),
])
show(p6)

<hr />

## Linking two plots

One of the best aspects of Bokeh is linking plots. We can link the brushing. This will allow you to select and pan with the plots both reacting to each other.

In [12]:

N = 300
x = np.linspace(0, 4*np.pi, N)
y1 = np.sin(x)
y2 = np.cos(x)

source = ColumnDataSource()
source.add(data=x, name='x')
source.add(data=y1, name='y1')
source.add(data=y2, name='y2')

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

s1 = figure(tools=TOOLS, plot_width=350, plot_height=350)
s1.scatter('x', 'y1', source=source)

# Linked brushing in Bokeh is expressed by sharing data sources between
# renderers. Note below that s2.scatter is called with the `source`
# keyword argument, and supplied with the same data source from s1.scatter
s2 = figure(tools=TOOLS, plot_width=350, plot_height=350, x_range=s1.x_range)
s2.scatter('x', 'y2', source=source, )

p = gridplot([[s1,s2]])
show(p)

<hr />

## Basic interactivity

Bokeh lets you use a Python update function to update your plots.

In IPython notebook we can use the interactive widgets provided by the notebook. One can also use Flask or Bokeh-server to embed in outside the notebook.

In [13]:
x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=y))

p = figure(title="simple line example", plot_height=300, plot_width=600)
p.line(x, y, color="#2222aa", line_width=3, source=source, name="foo")

In [14]:
from IPython.html.widgets import interact
@interact(f=["sin", "cos", "tan"], w=(0,100), A=(1,10), phi=(0, 10, 0.1))
def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    elif f == "tan": func = np.tan
    source.data['y'] = A * func(w * x + phi)
    source.push_notebook()


/Users/cdoig/anaconda/envs/py101/lib/python3.4/site-packages/ipykernel/__main__.py:8: BokehDeprecationWarning: bokeh.models.sources.push_notebook was deprecated in Bokeh 0.11.0; please use bokeh.io.push_notebook instead


In [15]:
show(p)

<hr/>

## [bokeh.charts](http://bokeh.pydata.org/docs/user_guide/charts.html)
Common Schemas for common tasks (and parameters).

Expects data to be formatted as either an OrderedDict or a pandas dataframe.

Supported Schemas: Bar, Boxplot, Categorical Heatmap, Histogram, Scatter, Timeseries

In [13]:
from collections import OrderedDict
from bokeh.charts import Histogram
from bokeh.io import show

mu, sigma = 0, 0.5
normal_dist = OrderedDict(normal=np.random.normal(mu, sigma, size=1000))
hist = Histogram(normal_dist, bins=50,
                 title="kwargs, dict_input", 
                 ylabel="frequency", 
                 width=400, 
                 height=350)

In [14]:
show(hist)